<a href="https://colab.research.google.com/github/ereinha/SineKAN/blob/main/SineKAN_MNIST_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sine_kan import *

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
from torchsummary import summary

In [ ]:
# Load MNIST
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))]
)
train_set = torchvision.datasets.MNIST(
    root="./data", train=True, download=True, transform=transform
)
val_set = torchvision.datasets.MNIST(
    root="./data", train=False, download=True, transform=transform
)
train_loader = DataLoader(train_set, batch_size=64, num_workers=2, shuffle=True)
val_loader = DataLoader(val_set, batch_size=64, num_workers=2, shuffle=False)

In [ ]:
epochs = 30
lrs = [1e-4, 2e-4, 3e-4, 4e-4, 5e-4]
gammas = [0.7, 0.8, 0.9]
hdims = [64, 128, 256]
best_accs = []
for lr in lrs:
    for gamma in gammas:
        for hdim in hdims:
            best_acc = 0
            # Define model
            model = SineKAN(layers_hidden=[28 * 28, hdim, 10], grid_size=8)
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            model.to(device)
            # Define optimizer
            optimizer = optim.AdamW(model.parameters(), lr=lr)
            # Define learning rate scheduler
            scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=gamma)

            # Define loss
            criterion = nn.CrossEntropyLoss()
            for epoch in range(epochs):
                # Train
                model.train()
                with tqdm(train_loader) as pbar:
                    for i, (images, labels) in enumerate(pbar):
                        images = images.view(-1, 28 * 28).to(device)
                        optimizer.zero_grad()
                        output = model(images)
                        loss = criterion(output, labels.to(device))
                        loss.backward()
                        optimizer.step()
                        accuracy = (output.argmax(dim=1) == labels.to(device)).float().mean()
                        pbar.set_postfix(loss=loss.item(), accuracy=accuracy.item(), lr=optimizer.param_groups[0]['lr'])

                # Validation
                model.eval()
                val_loss = 0
                val_accuracy = 0
                with torch.no_grad():
                    for images, labels in val_loader:
                        images = images.view(-1, 28 * 28).to(device)
                        output = model(images)
                        val_loss += criterion(output, labels.to(device)).item()
                        val_accuracy += (
                            (output.argmax(dim=1) == labels.to(device)).float().mean().item()
                        )
                val_loss /= len(val_loader)
                val_accuracy /= len(val_loader)
                if val_accuracy > best_acc:
                    best_acc = val_accuracy

                # Update learning rate
                scheduler.step()

                print(
                    f"Epoch {epoch + 1}, Val Loss: {val_loss}, Val Accuracy: {val_accuracy}"
                )
            best_accs.append(best_acc)
            print(f"LR: {lr} Gamma: {gamma} Hdim: {hdim} Best Accuracy: {best_acc}")

In [ ]:
hdims = [16, 32, 64, 128, 256, 512]
epochs = 30
val_accs = np.empty((6, epochs))
for h, hdim in enumerate(hdims):
    # Define model
    model = SineKAN(layers_hidden=[28 * 28, hdim, 10], grid_size=8)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    # Define optimizer
    optimizer = optim.AdamW(model.parameters(), lr=3e-4)
    # Define learning rate scheduler
    scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

    # Define loss
    criterion = nn.CrossEntropyLoss()
    for epoch in range(epochs):
        # Train
        model.train()
        with tqdm(train_loader) as pbar:
            for i, (images, labels) in enumerate(pbar):
                images = images.view(-1, 28 * 28).to(device)
                optimizer.zero_grad()
                output = model(images)
                loss = criterion(output, labels.to(device))
                loss.backward()
                optimizer.step()
                accuracy = (output.argmax(dim=1) == labels.to(device)).float().mean()
                pbar.set_postfix(loss=loss.item(), accuracy=accuracy.item(), lr=optimizer.param_groups[0]['lr'])

        # Validation
        model.eval()
        val_loss = 0
        val_accuracy = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images = images.view(-1, 28 * 28).to(device)
                output = model(images)
                val_loss += criterion(output, labels.to(device)).item()
                val_accuracy += (
                    (output.argmax(dim=1) == labels.to(device)).float().mean().item()
                )
        val_loss /= len(val_loader)
        val_accuracy /= len(val_loader)
        val_accs[h, epoch] = val_accuracy

        # Update learning rate
        scheduler.step()

        print(
            f"Epoch {epoch + 1}, Val Loss: {val_loss}, Val Accuracy: {val_accuracy}"
        )

In [ ]:
n_layers = [1, 2, 3, 4]
epochs = 30
layer_accs = np.empty((4, epochs))
for h, n_layer in enumerate(n_layers):
    # Define model
    model = SineKAN(layers_hidden=[28 * 28] + [128]*n_layer + [10], grid_size=8)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    # Define optimizer
    optimizer = optim.AdamW(model.parameters(), lr=3e-4)
    # Define learning rate scheduler
    scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

    # Define loss
    criterion = nn.CrossEntropyLoss()
    for epoch in range(epochs):
        # Train
        model.train()
        with tqdm(train_loader) as pbar:
            for i, (images, labels) in enumerate(pbar):
                images = images.view(-1, 28 * 28).to(device)
                optimizer.zero_grad()
                output = model(images)
                loss = criterion(output, labels.to(device))
                loss.backward()
                optimizer.step()
                accuracy = (output.argmax(dim=1) == labels.to(device)).float().mean()
                pbar.set_postfix(loss=loss.item(), accuracy=accuracy.item(), lr=optimizer.param_groups[0]['lr'])

        # Validation
        model.eval()
        val_loss = 0
        val_accuracy = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images = images.view(-1, 28 * 28).to(device)
                output = model(images)
                val_loss += criterion(output, labels.to(device)).item()
                val_accuracy += (
                    (output.argmax(dim=1) == labels.to(device)).float().mean().item()
                )
        val_loss /= len(val_loader)
        val_accuracy /= len(val_loader)
        layer_accs[h, epoch] = val_accuracy

        # Update learning rate
        scheduler.step()

        print(
            f"Epoch {epoch + 1}, Val Loss: {val_loss}, Val Accuracy: {val_accuracy}"
        )

In [ ]:
from torch.profiler import profile, record_function, ProfilerActivity
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def profile_model(model, inputs, num_runs=1000):
    total_times = []
    for _ in range(num_runs):
        with profile(activities=[ProfilerActivity.CUDA], record_shapes=True) as prof:
            model(inputs)
        total_time = 0
        # Extract total CUDA time spent on 'model_inference'
        for event in prof.key_averages():
            total_time += event.cuda_time_total
        total_times.append(total_time)
    mean = np.mean(total_times)
    std = np.std(total_times)
    print(f"Mean CUDA time: {mean} micros")
    print(f"Standard deviation of CUDA time: {std} micros")
    return mean, std

In [ ]:
means = []
stds = []
inputs = torch.randn((1, 784)).to(device)
hdims = [16, 32, 64, 128, 256, 512]
for h in hdims:
    model = SineKAN(layers_hidden=[28 * 28, h, 10], grid_size=8).to(device)
    print('Hidden Dims: %d' % h)
    mean, std = profile_model(model, inputs, 1000)
    means.append(mean)
    stds.append(std)

In [ ]:
means = []
stds = []
hdims = [16, 32, 64, 128, 256, 512]
for h in hdims:
    model = SineKAN(layers_hidden=[28 * 28, 128, 10], grid_size=8).to(device)
    inputs = torch.randn((h, 784)).to(device)
    print('Input Batch Size: %d' % h)
    mean, std = profile_model(model, inputs, 1000)
    means.append(mean)
    stds.append(std)

In [ ]:
means = []
stds = []
nlayers = [1, 2, 3, 4]
for n in nlayers:
    model = SineKAN(layers_hidden=[28 * 28] + [128]*n +[10], grid_size=8).to(device)
    inputs = torch.randn((1, 784)).to(device)
    print('Number of Hidden Layers: %d' % n)
    mean, std = profile_model(model, inputs, 1000)
    means.append(mean)
    stds.append(std)

In [ ]:
means = []
stds = []
grid_sizes = [2, 4, 6, 8, 10, 12]
for g in grid_sizes:
    model = SineKAN(layers_hidden=[28 * 28, 128, 10], grid_size=g).to(device)
    inputs = torch.randn((1, 784)).to(device)
    print('Grid Size: %d' % n)
    mean, std = profile_model(model, inputs, 1000)
    means.append(mean)
    stds.append(std)